<a href="https://colab.research.google.com/github/raul-lp/mia/blob/master/Practica_2_2_AlgoritmoA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Algoritmo A*
  Est_abiertos.insertar ( Estado inicial )
  Actual= Est_abiertos.primero ( )
  mientras (no es_final?( Actual )) y (no Est_abiertos.vacia?()) hacer
    Est_abiertos.borrar_primero ( )
    Est_cerrados.insertar ( Actual )
    hijos=generar_sucesores ( Actual )
    hijos=tratar_repetidos( Hijos, Est_cerrados, Est_abiertos )
    Est_abiertos.insertar ( Hijos )
    Actual=Est_abiertos.primero ( )
  fmientras
fAlgoritmo
```

In [ ]:
class Ciudad():

  def __init__(self, nombre='', padre=None, distancia_a_padre=0, n_nodo:int=1) -> None:
    self.nombre = nombre
    self.padre = padre
    self.distancia_a_padre = distancia_a_padre
    self.n_nodo = n_nodo
    self.heuristica = ()

  def mostrar(self):
    nodo_padre = 0
    if self.padre:
      nodo_padre = self.padre.n_nodo
    print(f'Ciudad: {self.nombre: <14} | Nº nodo: {self.n_nodo: >2} | Nodo Padre: {nodo_padre} | Heuristica: {self.heuristica} ')

class Diagrama:

  distancia_a_bucharest = {'Arad': 366, 'Bucharest': 0, 'Craiova': 160, 'Dobreta': 242, 'Eforie': 161, 'Fagaras': 176, 'Giurgiu': 77, 'Hirsova': 151, 'Iasi': 226, 'Lugoj': 244, 'Mehadia': 241, 'Neamt': 234, 'Oradea': 380, 'Pitesti': 10, 'Rimnicu_Vilcea': 193, 'Sibiu': 253, 'Timisoara': 329, 'Urziceni': 80, 'Vaslui': 199, 'Zerind': 374}
  distancia_a_ciudad_vecina = {
      'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
      'Bucharest': {'Urziceni': 85, 'Giurgiu': 90, 'Pitesti': 101},
      'Craiova': {'Pitesti': 138, 'Dobreta': 120, 'Rimnicu_Vilcea': 146},
      'Dobreta': {'Mehadia': 75, 'Craiova': 120},
      'Eforie': {'Hirsova': 86},
      'Fagaras': {'Bucharest': 211, 'Sibiu': 99},
      'Giurgiu': {'Bucharest': 90},
      'Hirsova': {'Eforie': 86, 'Urziceni': 98},
      'Iasi': {'Vaslui': 92, 'Neamt': 87},
      'Lugoj': {'Mehadia': 70, 'Timisoara': 111},
      'Mehadia': {'Dobreta': 75, 'Lugoj': 70},
      'Neamt': {'Iasi': 87},
      'Oradea': {'Sibiu': 151, 'Zerind': 71},
      'Pitesti': {'Bucharest': 101, 'Craiova': 138, 'Rimnicu_Vilcea': 97},
      'Rimnicu_Vilcea': {'Pitesti': 97, 'Craiova': 146, 'Sibiu': 80},
      'Sibiu': {'Fagaras': 99, 'Rimnicu_Vilcea': 80, 'Arad': 140, 'Oradea': 151},
      'Timisoara': {'Lugoj': 111, 'Arad': 118},
      'Urziceni': {'Vaslui': 142, 'Hirsova': 98, 'Bucharest': 85},
      'Vaslui': {'Urziceni': 142, 'Iasi': 92},
      'Zerind': {'Oradea': 71, 'Arad': 75}
  }

  def __init__(self, estado_inicial:Ciudad, estado_final:Ciudad):
    self.estado_inicial = estado_inicial
    self.estado_final = estado_final
    self.estado_actual = None
    self.estados_abiertos = []
    self.estados_cerrados = []
    self.n_nodos = 1

  def incrementarNodos(self) -> int:
    self.n_nodos += 1
    return self.n_nodos

  def calcularHeuristica(self, estado:Ciudad) -> tuple:
    d2b = self.distancia_a_bucharest[estado.nombre]
    i_h = d2b + estado.distancia_a_padre
    s_h = f'{d2b: >3} + {estado.distancia_a_padre: >3}'
    estado_padre = estado.padre
    while estado_padre != None:
      i_h += estado_padre.distancia_a_padre
      if estado_padre.distancia_a_padre > 0:
        s_h += f' + {estado_padre.distancia_a_padre: >3}'
      estado_padre = estado_padre.padre
    return (i_h, s_h)

  def ordenarEstados(self, estados:list) -> list:
    return sorted(estados, key=lambda estado: estado.heuristica[0])

  def insertarEstadosAbiertos(self, estados:list):
    self.estados_abiertos += estados
    self.estados_abiertos = self.ordenarEstados(self.estados_abiertos)

  def primeroEstadosAbiertos(self) -> Ciudad:
    return self.estados_abiertos[0]

  def borrarPrimeroEstadosAbiertos(self):
    self.estados_abiertos.pop(0)

  def insertarEstadosCerrados(self, estados:list):
    self.estados_cerrados += estados
    self.estados_cerrados = self.ordenarEstados(self.estados_cerrados)

  def generarSucesores(self, estado:Ciudad):
    hijos = []
    for ciudad, distancia in self.distancia_a_ciudad_vecina[estado.nombre].items():
      hijo = Ciudad(ciudad, estado, distancia, self.incrementarNodos())
      hijo.heuristica = self.calcularHeuristica(hijo)
      hijos += [hijo]
    hijos = self.ordenarEstados(hijos)
    return hijos

  def tratarRepetidos(self, hijos) -> list:
    for idx_hijos, hijo in enumerate(hijos):
      for idx_estado_abiertos, estado_abierto in enumerate(self.estados_abiertos):
        if hijo.nombre == estado_abierto.nombre and hijo.heuristica[0] < estado_abierto.heuristica[0]:
          self.estados_abiertos[idx_estado_abiertos] = hijo
          hijos.pop(idx_hijos)
    for idx_hijos, hijo in enumerate(hijos):
      for idx_estado_cerrado, estado_cerrado in enumerate(self.estados_cerrados):
        if hijo.nombre == estado_cerrado.nombre and hijo.heuristica[0] < estado_cerrado.heuristica[0]:
          self.insertarEstadosAbiertos(estado_cerrado)
          self.estados_cerrados.pop(idx_estado_cerrado)
    self.estados_abiertos = self.ordenarEstados(self.estados_abiertos)
    self.estados_cerrados = self.ordenarEstados(self.estados_cerrados)
    hijos = self.ordenarEstados(hijos)
    return hijos

while True:
  ciudad_inicial = input('Introduza la ciudad de origen: ')
  if ciudad_inicial in Diagrama.distancia_a_bucharest:
    break
  else:
    print('La ciudad de origen no es válida.')
print()
d = Diagrama(Ciudad(ciudad_inicial), Ciudad('Bucharest'))
d.estado_inicial.heuristica = d.calcularHeuristica(d.estado_inicial)

d.insertarEstadosAbiertos([d.estado_inicial])
d.estado_actual = d.primeroEstadosAbiertos()

print('Estado Inicial: ', end='')
d.estado_inicial.mostrar()
print('Estado Final:   ', end='')
d.estado_final.mostrar()

it = 0
while d.estado_actual.nombre != d.estado_final.nombre and len(d.estados_abiertos) > 0:
  it += 1
  print()
  print(f'Iteración {it}')
  print(f'Estado Actual: {d.estado_actual.nombre}')
  print()
  d.borrarPrimeroEstadosAbiertos()
  d.insertarEstadosCerrados([d.estado_actual])
  hijos = d.generarSucesores(d.estado_actual)
  hijos = d.tratarRepetidos(hijos)
  d.insertarEstadosAbiertos(hijos)
  d.estado_actual = d.primeroEstadosAbiertos()
  for hijo in hijos:
    hijo.mostrar()
  print()
  print(f'Estados Abiertos({len(d.estados_abiertos): >2}): {[estado_abierto.nombre for estado_abierto in d.estados_abiertos]}')
  print(f'Estados Cerrados({len(d.estados_cerrados): >2}): {[estado_cerrado.nombre for estado_cerrado in d.estados_cerrados]}')

Introduza la ciudad de origen: Arad

Estado Inicial: Ciudad: Arad           | Nº nodo:  1 | Nodo Padre: 0 | Heuristica: (366, '366 +   0') 
Estado Final:   Ciudad: Bucharest      | Nº nodo:  1 | Nodo Padre: 0 | Heuristica: () 

Iteración 1
Estado Actual: Arad

Ciudad: Sibiu          | Nº nodo:  3 | Nodo Padre: 1 | Heuristica: (393, '253 + 140') 
Ciudad: Timisoara      | Nº nodo:  4 | Nodo Padre: 1 | Heuristica: (447, '329 + 118') 
Ciudad: Zerind         | Nº nodo:  2 | Nodo Padre: 1 | Heuristica: (449, '374 +  75') 

Estados Abiertos( 3): ['Sibiu', 'Timisoara', 'Zerind']
Estados Cerrados( 1): ['Arad']

Iteración 2
Estado Actual: Sibiu

Ciudad: Rimnicu_Vilcea | Nº nodo:  6 | Nodo Padre: 3 | Heuristica: (413, '193 +  80 + 140') 
Ciudad: Fagaras        | Nº nodo:  5 | Nodo Padre: 3 | Heuristica: (415, '176 +  99 + 140') 
Ciudad: Arad           | Nº nodo:  7 | Nodo Padre: 3 | Heuristica: (646, '366 + 140 + 140') 
Ciudad: Oradea         | Nº nodo:  8 | Nodo Padre: 3 | Heuristica: (671, '380